In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelFinal & 'status="complete"'

In [4]:
@schema
class CoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.ProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.ProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.ProofreadLabel
    """
    
    key_source = pinky.ProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.ProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [5]:
start = time.time()
CoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349473583
Segment 648518346349473583 vertex labels regenerated in: 2.6436421871185303 seconds.
648518346349473597
Segment 648518346349473597 vertex labels regenerated in: 3.7163257598876953 seconds.
648518346349473821
Segment 648518346349473821 vertex labels regenerated in: 3.074528694152832 seconds.
648518346349475035
Segment 648518346349475035 vertex labels regenerated in: 4.266421556472778 seconds.
648518346349475518
Segment 648518346349475518 vertex labels regenerated in: 4.397952318191528 seconds.
648518346349475536
Segment 648518346349475536 vertex labels regenerated in: 4.077558517456055 seconds.
648518346349478248
Segment 648518346349478248 vertex labels regenerated in: 4.681041240692139 seconds.
648518346349478785
Segment 648518346349478785 vertex labels regenerated in: 4.809096336364746 seconds.
648518346349479706
Segment 648518346349479706 vertex labels regenerated in: 3.0928375720977783 seconds.
648518346349479929
Segment 648518346349479929 vertex labels regenerate

In [6]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()